In [15]:
import pandas as pd
import numpy as np
import os
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

### Settings

In [16]:
# Directories
out_dir = os.path.join(os.pardir,"output")
src = os.path.join(os.pardir, "src")

# Raw data
middle_income_file = os.path.join(src, "HE0110J7.xlsx") # Middle income
median_income_file = os.path.join(src, "HE0110J8.xlsx") # Median income

### Helpers

In [17]:
def checkDir(directory):
    if not os.path.exists(directory):
        print directory + " did not exist. Creating it..."
        os.makedirs(directory)
    else:
        print directory + " already exists. No further action."
    return

### Extract entities - municipalities

In [18]:
def extract_entities_municipalities(data):
    muni = data[["Unnamed: 0", "Unnamed: 1"]].copy() # Copy entity columns
    muni.rename(columns = {"Unnamed: 0": "municipality", "Unnamed: 1": "name"}, inplace=True) # Rename columns
    muni = muni.dropna() # Drop NaN rows
    muni["municipality"] = muni["municipality"].map(to_concept_id) # Make id string alphanumeric
    muni["county"] = muni["municipality"].astype(unicode).map(lambda x: x[:2]) # Add county
    muni["is--municipality"] = "TRUE"
    
    return muni

### Extract concepts

In [19]:
def extract_concepts(out_dir):
    concept_file = os.path.join(out_dir, "ddf--concepts.csv")
    
    concepts = ["Name", "Middle income", "Median income", "Year", "Municipality", "County"]
    df_con = pd.DataFrame([], columns = ["concept", "name", "concept_type"])

    df_con["name"] = concepts
    df_con["concept"] = df_con["name"].astype(unicode).map(to_concept_id)

    df_con["concept_type"] = "measure"
    df_con["concept_type"].iloc[0] = "string"
    df_con["concept_type"].iloc[3] = "time"
    df_con["concept_type"].iloc[4] = "entity_domain"
    df_con["concept_type"].iloc[5] = "entity_domain"
    
    # If not exist, create new
    # Else, add to existing file
    if os.path.isfile(concept_file):
        print concept_file + " exists. Adding new entries."
        data = pd.read_csv(concept_file, encoding="utf-8")
        df_con = pd.concat([data, df_con])
        df_con = df_con.drop_duplicates(subset=["concept"])
        
    return df_con

### Extract datapoints

In [20]:
def extract_datapoints(data, measure):
    dps = data.copy()
    dps.rename(columns = {"Unnamed: 0": "municipality", "Unnamed: 1": "name"}, inplace=True)

    income = dps[dps.columns[2:]]
    entities = dps[["municipality", "name"]]
    
    datapoints = pd.DataFrame([], columns=["municipality", "name", "year", measure])
    for year in income.columns:
        tmp = entities.copy()
        tmp["year"] = year
        tmp[measure] = income[year]*1000
        datapoints = pd.concat([datapoints,tmp])
        
    datapoints["municipality"] = datapoints["municipality"].astype(unicode).map(to_concept_id)
    
    return datapoints.sort_values(by=["municipality","year"])

### Main script

In [21]:
if __name__ == "__main__":
    
    #Check if output dir exists, otherwise create it
    checkDir(out_dir)
    
    # Load raw data
    files = [middle_income_file, median_income_file]
    measure = ["middle_income", "median_income"]
    first = True
    
    for i, infile in enumerate(files):
        print "Reading " + infile
        data = pd.read_excel(infile, skiprows=[0,1], parse_cols="A,B,G:V", skip_footer=41, \
                                 converters={'Unnamed: 0': lambda x: str(x)})
        
        data.replace(unicode(".."),0,inplace=True)
    
        if (first):
            # Extract entities (counties)
            muni = extract_entities_municipalities(data)
            path = os.path.join(out_dir, "ddf--entities--municipality.csv")
            print "Printing " + path
            muni.to_csv(path, index=False, encoding="utf-8")

            # Extract concepts
            concepts = extract_concepts(out_dir)
            path = os.path.join(out_dir, "ddf--concepts.csv")
            print "Printing " + path
            concepts.to_csv(path, index=False, encoding="utf-8")

            first = False
    
        # Extract datapoints
        datapoints = extract_datapoints(data, measure[i])
        path = os.path.join(out_dir, "ddf--datapoints--{}--by--municipality--year.csv".format(measure[i]))
        print "Printing " + path
        datapoints.to_csv(path, index=False, encoding="utf-8")
        
    # Create index file
    print("Creating index files...")
    create_index_file(out_dir)
    
    del muni, concepts, data, datapoints    

../output already exists. No further action.
Reading ../src/HE0110J7.xlsx
Printing ../output/ddf--entities--municipality.csv
../output/ddf--concepts.csv exists. Adding new entries.
Printing ../output/ddf--concepts.csv
Printing ../output/ddf--datapoints--middle_income--by--municipality--year.csv
Reading ../src/HE0110J8.xlsx
Printing ../output/ddf--datapoints--median_income--by--municipality--year.csv
Creating index files...
